# Practical Statistics for Data Science

### Estructura de datos no rectangular

Estos son datos como:
- Series temporales los cuales registran mediciones sucesivas de la misma variable. Es el mmaterial ideal para pronosticos estadisticos y el componente principal del Internet de las Cosas.
- Datos espaciales utilizados en mapas y análisis de ubicación y son mas complejas y variadas que las estructuras de datos rectangulares. El foco de los datos es un objeto y sus coordenadas espaciales.
- Los grafos son estructuras de datos que se utilizan para representar relaciones fisicas, sociales y abstractas. Un ejemplo son las conexiones entre personas en la Red de Facebook.Utilizados en la optimizacion de la red  y en sistemas de recomendacion.

La estructura de datos basica in Ciencia de datos es la matriz rectangular en la cual las filas son registros y las columnas son las variables.

### Estimacion de Ubicacion (Tendencia Central)

Un paso tipico a la hora de explorar la data es obtener el valor tipico de cada variable que nos conduce a estimar en donde se encuentra la mayoria de los datos

#### Ejemplo: Estimaciones de ubicacion de poblacion y tasa de homicidios 

In [1]:
import pandas as pd

In [6]:
state = pd.read_csv('./data/state.csv') 
print(state.head(8))

         State  Population  Murder.Rate Abbreviation
0      Alabama     4779736          5.7           AL
1       Alaska      710231          5.6           AK
2      Arizona     6392017          4.7           AZ
3     Arkansas     2915918          5.6           AR
4   California    37253956          4.4           CA
5     Colorado     5029196          2.8           CO
6  Connecticut     3574097          2.4           CT
7     Delaware      897934          5.8           DE
